In [1]:
"""
The code for testing the classifiers. Uses binary.csv and multiclass.csv

Loads feature lists and tuning ranges from pickles in CWD.

Change model types in models to tune on specific ones.

Writes output files.

Author: Wesley
"""

# Accelerates tuning of some classifiers
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import numpy as np

from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real

from sklearn.feature_selection import RFECV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import pickle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score
)

from time import time

from sklearn.metrics import classification_report

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
multiclass = pd.read_csv("multiclass_train.csv")
multiclass_test = pd.read_csv("multiclass_test.csv")

Preprocessing (make labels numeric)

In [3]:
# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()
multiclass[" Label"] = le.fit_transform(multiclass[" Label"].values)
multiclass_test[" Label"] = le.transform(multiclass_test[" Label"].values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multiclass[' Label']))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load feature sets and search spaces and enumerate their contents.

In [4]:
feature_sets = pickle.load(open("feature_sets.pickle", 'rb'))
search_spaces = pickle.load(open("hyperparameter_search_spaces.pickle", 'rb'))

print(f"Available Tuning Ranges: {search_spaces.keys()}")

print("Feature Sets for Binary Dataset:")
for key, value in feature_sets["Binary"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")
        
    else:
        print(key)

print("Feature Sets for Multiclass Dataset:")
for key, value in feature_sets["Multiclass"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")

    else:
        print(key)

Available Tuning Ranges: dict_keys(['XGBoost', 'Bagging SVM', 'SVC (RBF)', 'SVC (Poly)', 'Logistic Regression', 'Random Forest', 'KNN', 'Linear SVC', 'Naive Bayes', 'Decision Tree'])
Feature Sets for Binary Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Linear SVC', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95
Feature Sets for Multiclass Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95


In [5]:
"""
This is a helper method to place our performance results in a DataFrame for future analysis.
"""
def format_results_multiclass(y_test, predicted_values, fold_index, fitTime):
    # get scores
    accuracy = accuracy_score(y_test,predicted_values)
    recall_pos = recall_score(y_test, predicted_values, average='macro')
    precision_pos = precision_score(y_test,predicted_values, average='macro')
    f1 = f1_score(y_test,predicted_values, average='macro')

    cols = ["Fitting Time", "accuracy", "Precision", "Recall", "F1 Score"]
    results = [fitTime, accuracy, precision_pos, recall_pos, f1]

    outFrame = pd.DataFrame([results], columns=cols, index=[fold_index])

    return outFrame

This is for tuning on the multiclass set

In [6]:
models = {
    #"Decision Tree": DecisionTreeClassifier(random_state=42),
    #"Random Forest": RandomForestClassifier(random_state=42),
    #"XGBoost": xgb.XGBClassifier(random_state=42, num_class=12, objective='multi:softmax'),
    #"Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
    #"KNN": KNeighborsClassifier(),
    #"Naive Bayes": GaussianNB(),
    #"SVC (RBF)": make_pipeline(StandardScaler(), SVC(random_state=42, kernel='rbf')),
}

score_methods = ['accuracy']

feature_set = feature_sets["Multiclass"]

y = multiclass[" Label"].copy()
X = multiclass.drop([" Label"], axis=1)

y_test_f = multiclass[" Label"].copy()
X_test_f = multiclass.drop([" Label"], axis=1)

In [7]:
params = []

# This will hold all of our results.
runFrame = None

for name, model in models.items():
        for feature_key, feature_val in feature_set.items():

            # If we're on the RFE sets, check if we have one for this classifier. If not, skip it.
            if feature_key == "RFE Sets":
                if name in feature_val.keys():
                    feature_val = feature_val[name]
                else:
                    continue
            
            for score_method in score_methods:
                opt = BayesSearchCV(estimator=model,search_spaces=search_spaces[name],n_iter=50,scoring=score_method,cv=5,n_jobs=5)
                kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

                counter = 0

                # Used to hold data for a single run (performance metric)
                perfFrame = None

                for train_index, test_index in kf.split(X, y):

                    counter += 1

                    # PCA requires different logic to create X.
                    if feature_key != "PCA":
                        current_X = X.loc[:, feature_val]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        current_X_f = X_test_f.loc[:, feature_val]
                        
                    else:
                        pca_trans = PCA(n_components=feature_val, random_state=42)
                        current_X = X.loc[:, X.columns]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        # Apply PCA to training set and use it to transform test set.
                        X_train = pca_trans.fit_transform(X_train)
                        X_test = pca_trans.transform(X_test)
                        current_X_f = pca_trans.transform(X_test_f)

                        # Convert back to DataFrames
                        pca_cols = ["PC"+str(i) for i in list(range(1, len(X_train[0])+1))]
                        X_train = pd.DataFrame(data=X_train, columns=pca_cols)
                        X_test = pd.DataFrame(data=X_test, columns=pca_cols)
                        current_X_f = pd.DataFrame(data=current_X_f, columns=pca_cols)

                    startTime = time()

                    opt.fit(X_train,Y_train)

                    endTime = time()
                    fitTime = endTime - startTime

                    predicted_values = opt.predict(X_test)

                    # get metrics for this fold.
                    foldFrame = format_results_multiclass(Y_test, predicted_values, counter, fitTime)

                    # Add them to our lists of metric.
                    if perfFrame is None:
                        perfFrame = foldFrame
                    else:
                        perfFrame = pd.concat([perfFrame, foldFrame])

                    # Print a classification report on the testing results.
                    print("Validation Results: ")
                    print(classification_report(Y_test, predicted_values, target_names=multiclass_labels, digits=6))

                    print("Testing Results: ")
                    # Print a classification report on the testing results.
                    pred_test = opt.predict(current_X_f)
                    print(classification_report(y_test_f, pred_test, target_names=multiclass_labels, digits=6))

                    # Add tuple with the best params as well as the related model/config
                    params.append((f"Multiclass {name} {feature_key} {score_method} Fold {counter}", opt.best_params_))

                # Create a new line in the results table that averages all the folds
                perfFrame.loc["fold average"] = perfFrame.mean()

                # Mark the results table with the chosen classifier and the current performance metric.
                perfFrame['metric'] = [score_method for j in range(0,6)]
                perfFrame['Classifier'] = [name for j in range(0,6)]
                perfFrame['Feature Set'] = [feature_key for j in range(0,6)]
                perfFrame['Dataset'] = ["multiclass" for j in range(0,6)]
                print(f"{name} with {feature_key} and {score_method} completed.")

                # Add this run to the table with all runs.
                if runFrame is None:
                    runFrame = perfFrame
                else:
                    runFrame = pd.concat([runFrame, perfFrame])

# Write output file, best parameters, and best models to be used later.
runFrame.to_csv(f"multiclass_results_LR_{time()}.csv")
pickle.dump(params, open(f"multiclass_params_LR_{time()}.pickle", "wb"))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Results: 
              precision    recall  f1-score   support

         DNS   0.351789  0.154856  0.215048      2667
        LDAP   0.489086  0.680540  0.569144      2667
       MSSQL   0.580098  0.885596  0.701010      2666
         NTP   0.987914  0.950488  0.968840      2666
     NetBIOS   0.521708  0.274944  0.360108      2666
     Portmap   0.499277  0.777194  0.607981      2666
        SNMP   0.624379  0.706787  0.663032      2667
        SSDP   0.456349  0.172478  0.250340      2667
         Syn   0.989761  0.543682  0.701839      2667
        TFTP   0.997211  0.536183  0.697391      2667
         UDP   0.475989  0.802775  0.597627      2667
     UDP-lag   0.644286  0.845836  0.731430      2666

    accuracy                       0.610925     31999
   macro avg   0.634821  0.610947  0.588649     31999
weighted avg   0.634819  0.610925  0.588636     31999

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.359823  0.158929  0.22

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Results: 
              precision    recall  f1-score   support

         DNS   0.152623  0.036009  0.058270      2666
        LDAP   0.490780  0.698538  0.576512      2667
       MSSQL   0.572353  0.928732  0.708238      2666
         NTP   0.982080  0.945611  0.963501      2666
     NetBIOS   0.539694  0.277944  0.366923      2666
     Portmap   0.506015  0.788526  0.616444      2667
        SNMP   0.564456  0.737158  0.639350      2667
        SSDP   0.481457  0.535433  0.507012      2667
         Syn   0.984343  0.542182  0.699226      2667
        TFTP   0.998579  0.526809  0.689740      2667
         UDP   0.488131  0.424072  0.453852      2667
     UDP-lag   0.644729  0.848837  0.732837      2666

    accuracy                       0.607488     31999
   macro avg   0.617103  0.607488  0.584325     31999
weighted avg   0.617109  0.607488  0.584328     31999

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.134075  0.030751  0.05

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         DNS   0.160000  0.003900  0.007615     13333
        LDAP   0.440877  0.725118  0.548352     13333
       MSSQL   0.328689  0.633466  0.432806     13333
         NTP   0.075203  0.008326  0.014992     13332
     NetBIOS   0.000000  0.000000  0.000000     13333
     Portmap   0.417789  0.779269  0.543951     13333
        SNMP   0.563193  0.597960  0.580056     13332
        SSDP   0.057377  0.002625  0.005021     13332
         Syn   0.240663  0.999025  0.387886     13333
        TFTP   0.000000  0.000000  0.000000     13333
         UDP   0.065870  0.013650  0.022614     13333
     UDP-lag   0.006202  0.005925  0.006061     13333

    accuracy                       0.314107    159993
   macro avg   0.196322  0.314105  0.212446    159993
weighted avg   0.196321  0.314107  0.212446    159993



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

Validation Results: 
              precision    recall  f1-score   support

         DNS   0.227273  0.005624  0.010977      2667
        LDAP   0.443602  0.724306  0.550221      2666
       MSSQL   0.336850  0.646419  0.442903      2667
         NTP   0.065744  0.007124  0.012855      2667
     NetBIOS   0.000000  0.000000  0.000000      2667
     Portmap   0.438498  0.762191  0.556712      2666
        SNMP   0.565402  0.598275  0.581374      2666
        SSDP   0.052632  0.002251  0.004317      2666
         Syn   0.201697  0.998875  0.335623      2666
        TFTP   0.423780  0.052138  0.092852      2666
         UDP   0.070299  0.014998  0.024722      2667
     UDP-lag   0.000000  0.000000  0.000000      2667

    accuracy                       0.317645     31998
   macro avg   0.235481  0.317683  0.217713     31998
weighted avg   0.235459  0.317645  0.217688     31998

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.166667  0.003900  0.00

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
best = pickle.load(open("multiclass_params_LR_1670732570.800041.pickle", 'rb'))

In [8]:
final_feature_set = None
for key, value in feature_sets["Multiclass"].items():
    if key == "RFE Sets":
        final_feature_set = value["Logistic Regression"]
        break

print(final_feature_set)

X_fin = X.loc[:, final_feature_set]
x_test_fin = X_test_f.loc[:, final_feature_set]

[' Protocol', ' Flow Duration', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min']


In [12]:
for i in range(16, 20):
    print(best[i])

('Multiclass Logistic Regression RFE Sets accuracy Fold 2', OrderedDict([('logisticregression__C', 1368.724871427079), ('logisticregression__max_iter', 279)]))
('Multiclass Logistic Regression RFE Sets accuracy Fold 3', OrderedDict([('logisticregression__C', 9956.802629520667), ('logisticregression__max_iter', 214)]))
('Multiclass Logistic Regression RFE Sets accuracy Fold 4', OrderedDict([('logisticregression__C', 4602.593009396766), ('logisticregression__max_iter', 240)]))
('Multiclass Logistic Regression RFE Sets accuracy Fold 5', OrderedDict([('logisticregression__C', 2358.2284838718524), ('logisticregression__max_iter', 224)]))


In [18]:
clf = make_pipeline(StandardScaler(), LogisticRegression(random_state=42, C = 4602.593009396766, max_iter = 240))
clf.fit(X_fin, y)
pred_test = clf.predict(x_test_fin)
print(classification_report(y_test_f, pred_test, target_names=multiclass_labels, digits=6))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         DNS   0.127860  0.030601  0.049383     13333
        LDAP   0.488660  0.691667  0.572706     13333
       MSSQL   0.591760  0.902798  0.714914     13333
         NTP   0.984851  0.941119  0.962488     13332
     NetBIOS   0.507321  0.252081  0.336807     13333
     Portmap   0.496927  0.782270  0.607773     13333
        SNMP   0.552258  0.722922  0.626169     13332
        SSDP   0.463269  0.525053  0.492230     13332
         Syn   0.987230  0.545039  0.702329     13333
        TFTP   0.990220  0.539188  0.698198     13333
         UDP   0.480260  0.454361  0.466952     13333
     UDP-lag   0.648067  0.847596  0.734523     13333

    accuracy                       0.602889    159993
   macro avg   0.609890  0.602891  0.580373    159993
weighted avg   0.609889  0.602889  0.580371    159993

